## Training data Generation

13 params to run the ROI generation script:
- h5path : HDF5 file path
- Abhome : Home directory for "train" foler, should be inside the Ab folder
- hiclip_val : Target channel, higher limit for clipping
- lowclip_val : Target channel, lower limit for clipping
- nuc_clip_low   : Nuc channel, lower limit for clipping
- nuc_clip_high  : Nuc channel, higher limit for clipping
- cyto_clip_low  : Cyto channel, lower limit for clipping
- cyto_clip_high : Cyto channel, higher limit for clipping
- xstart : x-coordinate for selected ROI
- ystart : y-coordinate for selected ROI
- zstart : z-coordinate for selected ROI
- ROI_dim : Dimension of ROI to be cropped, 512 x 512 (default)
- no_of_layer : Number of layers/levels to sample, 50 (default)

Return: 
* "Train" folder in specified Antibody directory, with "Ch0", "Ch1", "Ch2" as subfolder, 
* Cropped ROIs blocks are saved in this folder w/ this format: *OTLS4_NODO_mmddyy_xxxxx_Xpos_xxxxx_xxxxx_Ypos_xxxxx_xxxxx_stack_xxxxx_xxxxx_mirror* 

In [8]:
import os
import h5py as h5
from CollectImgStackFused import *
import warnings
warnings.filterwarnings("ignore")

Define the main function that generates training ROIs.

`main()` : Grab the 13 input parameters to generate training ROIs. Create "train" folder. 

`CollectImgStackFused()` : Nested function.
- Load .h5 file with corresponding coords, rescale intensity and save it as 8-bits .jpeg stacks.
- Output it to "train" folder, and /ch0, /ch1, /ch2 subfolders. 

In [9]:
def main(h5path
        ,train_home
        ,hiclip_val
        ,lowclip_val
        ,nuc_clip_low
        ,nuc_clip_high
        ,cyto_clip_low
        ,cyto_clip_high
        ,clahe
        ,xstart
        ,ystart
        ,zstart
        ,ROI_dim
        ,no_of_layer
        ,orient
        ,res
        ):

    filename = h5path
    # savedir = os.getcwd() + os.sep + 'train'
    if not os.path.exists(train_home):
        os.mkdir(train_home)
    savedir = train_home + os.sep + 'train'
    if not os.path.exists(savedir):
        os.mkdir(savedir)

    split = filename.count("/") - 1
    blockname = filename.split("/")[split]
    print(blockname,"[ xyz:",xstart,ystart,zstart,"]")
    print(savedir)

    xstop = xstart + ROI_dim
    xcoords = (xstart, xstop)

    ystop = ystart + ROI_dim
    ycoords = (ystart, ystop)

    zstop = zstart + no_of_layer
    zcoords = (zstart, zstop)

    with h5.File(filename, 'r') as f:
        collectImgStackFused(f,
                        savedir,
                        blockname,
                        zcoords,
                        xcoords,
                        ycoords,
                        orient,
                        CLAHE=clahe,
                        hiclip_val  = hiclip_val,
                        lowclip_val = lowclip_val,
                        nuc_clip_low   = nuc_clip_low,
                        nuc_clip_high  = nuc_clip_high,
                        cyto_clip_low  = cyto_clip_low,
                        cyto_clip_high = cyto_clip_high, 
                        res = res)
    f.close()

**Auto Set up with GUI [Preferred]** 

You should already have the *ROI_coords_yyyy-mm-dd.csv* file ready. If not, use *ROI_v2.py* to export all the params into a .csv file.

In [11]:
import pandas as pd
import time
fname = "../coords/ROI_coords_2024-01-17.csv" #change this
res = 0 # change to 0 if you want 2X ds

df = pd.read_csv(fname)
df = df.dropna(how="all")


for i in range(len(df)):
    print("ROI #", i)
    if i == 0:
        start = time.time()
    elif i == 1:
        print("Time per ROI : ", start-time.time()/60, "min")

    h5path = df["h5path"][i]
    Ab_home = df["Abhome"][i]
    hiclip_val = df["pgp_clipHigh"][i]
    lowclip_val = df["pgp_clipLow"][i]
    nuc_clip_low = df["nuc_clipLow"][i]
    nuc_clip_high = df["nuc_clipHigh"][i]
    cyto_clip_low = df["cyto_clipLow"][i]
    cyto_clip_high = df["cyto_clipHigh"][i]
    if df["pgp_ctehmt_method"][i] == "Rescale":
        clahe = False
    else:
        clahe = True    
    xstart = df["xcoord"][i]
    ystart = df["ycoord"][i]
    zstart = df["zcoord"][i]
    ROI_dim = df["ROIdim"][i]
    no_of_layer = df["No_ofLayers"][i]
    orient = df["orient"][i]

    main(h5path
        ,Ab_home
        ,hiclip_val,lowclip_val
        ,nuc_clip_low,nuc_clip_high
        ,cyto_clip_low,cyto_clip_high
        ,clahe
        ,xstart,ystart,zstart
        ,ROI_dim,no_of_layer
        ,orient
        ,res)


ModuleNotFoundError: No module named 'pandas'

**Manual Setup**

you could preview to find the coords with bigstitcher or GUI

In [12]:
# change this
h5path = "W:/Trilabel_Data/CD31/Dec/OTLS4_NODO_01-03-24_PC_CD31/data-f0.h5"
Ab_home = os.getcwd()

hiclip_val = 2200
lowclip_val = 120
nuc_clip_low = 10
nuc_clip_high = 10000
cyto_clip_low = 10
cyto_clip_high = 5000
clahe = False

xstart = 200
ystart = 2500
zstart = 30
ROI_dim = 512
no_of_layer = 50
orient = 0 # 1 for xzy, 0 for zxy

main(h5path
    ,Ab_home
    ,hiclip_val,lowclip_val
    ,nuc_clip_low,nuc_clip_high
    ,cyto_clip_low,cyto_clip_high
    ,clahe
    ,xstart,ystart,zstart
    ,ROI_dim, no_of_layer
    ,orient)

TypeError: main() missing 1 required positional argument: 'res'